# Example Workflow for Normalizing and Validating δ15N Data from IRMS


## 1. SETUP

Define or import reference materials with known $\delta ^{15}N$ values for normalization and validation.


In [1]:
from isotools import USGS32_N2, USGS34_N2, USGS35_N2, ReferenceMaterial

# Custom reference materials can also be defined as needed
# USGSXXX = ReferenceMaterial("USGSXXX", true_value, uncertainty)

# for common use cases, materials can be added in isotools/standards.py

C:\Users\Damm\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## 2. Ingest


In [2]:
from isotools import NitrogenProcessor

# Processor super class manages data cleaning, peak selection
# Implemented as a stateful object for future extensions in more complex workflows

# In this example, we exclude row 26 (a botched injection)
proc = NitrogenProcessor(exclude_rows=[26])

# Load the data from an Isodat report
# Some fields are mandatory for the processor to work properly
# Expecting one sheet per isotope for multiple isotopes runs (IE: NCS)
# Expecting "Peak Nr" to be consistent between runs, to discriminate between reference and sample peaks

df = proc.load_data(r"DATA\nitrate 26112025.xls", sheet_name="n_damian.wke")
df.head()

,Row,Identifier 1,Identifier 2,Amount,Comment,Peak Nr,Ampl 28,Ampl 29,Area 28,Area 29,R 15N/14N,d 15N/14N
3,10,nitrato ES3,NaN,0.558,NaN,2,5055,3592,71.990,0.511,0.003622,-15.346
5,11,nitrato ES3,NaN,0.588,NaN,2,5445,3867,78.306,0.556,0.003622,-15.414
7,12,nitrato ES3,NaN,0.592,NaN,2,5314,3774,77.983,0.554,0.003622,-15.287
9,13,nitrato ES3,NaN,0.559,NaN,2,4891,3474,71.222,0.506,0.003622,-15.412
11,14,nitrato ES6,NaN,0.487,NaN,2,5684,4128,83.973,0.610,0.003702,6.575


In [3]:
## .aggregate to aggregae replicate measurements

stats_df = proc.aggregate(df)
stats_df

,mean,sem,count
Identifier 1,,,
USGS32,188.14050,0.182134,4
USGS34,5.17925,0.023859,4
USGS35,9.80700,0.015716,3
nitrato ES3,-15.36475,0.030352,4
nitrato ES6,5.32700,0.866005,4


In [4]:
## .process_file to do full processing in one step

stats_df = proc.process_file(r"DATA\nitrate 26112025.xls", sheet_name="n_damian.wke")
stats_df

,mean,sem,count
Identifier 1,,,
USGS32,188.14050,0.182134,4
USGS34,5.17925,0.023859,4
USGS35,9.80700,0.015716,3
nitrato ES3,-15.36475,0.030352,4
nitrato ES6,5.32700,0.866005,4


## 3. Normalization and uncertainty calculation


In [6]:
from isotools import Calibrator, TwoPointStrategy

# Define calibration strategy
strategy = TwoPointStrategy()

# Initialize calibrator with strategy
calib = Calibrator(strategy=strategy)

# Perform normalization of the aggregate data
results = calib.calibrate(stats_df, standards=[USGS32_N2, USGS34_N2])
results

[CalculationResult(identifier='USGS32', raw_mean=188.1405, corrected_delta=180.0, combined_uncertainty=1.0322019540942766, calibrated_with='USGS32/USGS34'),
 CalculationResult(identifier='USGS34', raw_mean=5.17925, corrected_delta=-1.8000000000000096, combined_uncertainty=0.20279102365571416, calibrated_with='USGS32/USGS34'),
 CalculationResult(identifier='USGS35', raw_mean=9.807, corrected_delta=2.79837779857756, combined_uncertainty=0.19859695276391945, calibrated_with='USGS32/USGS34'),
 CalculationResult(identifier='nitrato ES3', raw_mean=-15.36475, corrected_delta=-22.213607799465745, combined_uncertainty=0.25320463991799796, calibrated_with='USGS32/USGS34'),
 CalculationResult(identifier='nitrato ES6', raw_mean=5.327, corrected_delta=-1.6531877651688633, combined_uncertainty=0.8837267971862558, calibrated_with='USGS32/USGS34')]